# Свёрточная нейронная сеть прямого распространения (Convolutional Neural Network)

Это особый вид нейронной сети, предназначенной для обработки структурированных данных, таких как изображения и видео, и других данных сетчатой структуры. Они широко используются в области компьютерного зрения для решения задач, таких как классификация, обнаружение объектов и сегментация.

**Ключевые особенности и характеристики Feed forward CNN:**

1. **Свёрточные слои (Convolutional Layers):**

    ConvNet включает в себя сверточные слои, которые применяют фильтры (ядра) к входным данным. Эти фильтры скользят по входным данным и извлекают локальные признаки, такие как грани, текстуры и формы.

2. **Пулинг-слои (Pooling Layers):**

    Пулинг-слои используются для уменьшения размерности данных, уменьшая пространственное разрешение карт признаков. Они выполняют операции, такие как максимум или среднее, чтобы сохранить самые важные признаки.
    
3. **Полносвязные слои (Fully Connected Layers):**

    После сверточных и пулинг-слоев, один или несколько полносвязных слоев могут использоваться для выполнения окончательных предсказаний. Эти слои обрабатывают высокоуровневые признаки, полученные из предыдущих слоев.
    
4. **Функции активации:**

    Обычно в сверточных и полносвязных слоях применяются функции активации, такие как ReLU (Rectified Linear Unit), для введения нелинейности в сеть.
    
5. **Выходной слой (Output Layer):**

    Выходной слой предоставляет окончательные предсказания сети. В задачах классификации, это может быть слой с несколькими выходами, с активацией softmax для получения вероятностей классов.
    
6. **Функция потерь (Loss Function):**

    Функция потерь определяет, как сеть оценивает ошибку между предсказанными и фактическими значениями. Для задач классификации обычно используется перекрестная энтропия.
    
7. **Обучение и оптимизация:**

    ConvNet обучается с использованием алгоритмов оптимизации, таких как стохастический градиентный спуск (SGD) или его модификации, для минимизации функции потерь.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot  as plt

# Device configuration
device = 'cpu'# torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 1 Этап - подготовка данных!

In [2]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [3]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

# 2 Этап - Нейронная сеть!

In [4]:
# Fully connected neural network with one hidden layer
class Lenet5(nn.Module):
    def __init__(self, hidden_size1=500, num_classes=10):   # Количество нейронов в скрытом слое (hidden_size1)
                                                            # Количество классов для задачи классификации
        super().__init__()
        # self.conv1 и self.conv2 – два свёрточных слоя, которые выполняют свёртку и применяют функцию активации ReLU
        # self.batchnorm1 и self.batchnorm2 – слои нормализации данных для стабилизации обучения
        # self.maxpool – слой максимальной подвыборки (пулинга) для уменьшения размерности данных
        # self.fc1 и self.fc2 – два полносвязных слоя для окончательных предсказаний
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=0)
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1, padding=0)
        self.batchnorm2 = nn.BatchNorm2d(50)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(50 * 4 * 4, hidden_size1) # fully connected
        self.fc2 = nn.Linear(hidden_size1, num_classes)  
    
    def forward(self, x): # вход размера 1 x 28 x 28
        
        # Изображение x проходит через свёрточные слои, нормализацию, функции активации и пулинг
        out = self.conv1(x) # на выходе размер 20 x 24 x 24
        out = self.batchnorm1(out)
        out = self.maxpool(out) # на выходе размер 20 x 12 x 12
        out = self.relu(out)
        out = self.conv2(out)  # на выходе размер 50 x 8 x 8
        out = self.batchnorm2(out)
        out = self.maxpool(out) # на выходе размер 50 x 4 x 4
        out = self.relu(out)
        
        # Данные выравниваются в одномерный вектор и проходят через полносвязные слои
        # На выходе получаются вероятности принадлежности к каждому из num_classes классов
        out = out.reshape(-1, 50 * 4 * 4) # на выходе размер 800
        out = self.fc1(out) # на выходе размер 500
        out = self.relu(out)
        out = self.fc2(out) # на выходе размер 10
        
        return out

model = Lenet5(hidden_size1=500, num_classes=10).to(device)

In [5]:
# Блок нейронной сети, который инициализирует два слоя: полносвязный слой (nn.Linear) и слой активации ReLU (nn.ReLU)
class NN_block(nn.Module):
    def __init__(self, input_size):
        self.fc = nn.Linear(input_size, input_size)
        self.activation = nn.ReLU()
    def forward(self, x):
        out = x
        out = self.fc(out)
        out = self.activation(out)
        return out

# Нейронная сеть, которая состоит из блока NN_block, применённого несколько раз
class NN(nn.Module):
    def __init__(self, input_size):
        self.block = NN_block(input_size)

    def forward(self, x):
        out = x
        for i in range(10):
            out = self.block(out)
        return out

In [6]:
model

Lenet5(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (batchnorm1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (batchnorm2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# 3 этап - Loss-функция и оптимизатор

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# for p in model.parameters():
#     print(p)

In [9]:
model.conv1

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

In [10]:
model.conv1.weight.shape

torch.Size([20, 1, 5, 5])

In [11]:
model.conv1.weight[0]

tensor([[[ 0.1941, -0.0517, -0.0519,  0.1755,  0.1302],
         [ 0.0574, -0.1911,  0.1383,  0.0959,  0.1407],
         [ 0.1911, -0.1702,  0.1721,  0.0114,  0.0243],
         [-0.0407,  0.0872,  0.0679, -0.0665, -0.0050],
         [ 0.1138, -0.0689,  0.1730,  0.0792,  0.0481]]],
       grad_fn=<SelectBackward0>)

In [12]:
model.conv1.bias

Parameter containing:
tensor([-0.0746,  0.1342, -0.0511,  0.1069,  0.1199,  0.1945, -0.1893, -0.0013,
        -0.0530,  0.0713, -0.1644, -0.1431, -0.1650,  0.1569, -0.1283, -0.0246,
        -0.1610,  0.1596,  0.0253, -0.0365], requires_grad=True)

# 4 этап - обучение нейросети

In [13]:
for i, (images, labels) in enumerate(train_loader):  
    break
images.shape, labels.shape

(torch.Size([100, 1, 28, 28]), torch.Size([100]))

In [14]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1084
Epoch [1/5], Step [200/600], Loss: 0.1674
Epoch [1/5], Step [300/600], Loss: 0.0899
Epoch [1/5], Step [400/600], Loss: 0.0852
Epoch [1/5], Step [500/600], Loss: 0.0884
Epoch [1/5], Step [600/600], Loss: 0.0455
Accuracy of the network on the 10000 test images: 98.35 %
Epoch [2/5], Step [100/600], Loss: 0.0413
Epoch [2/5], Step [200/600], Loss: 0.0314
Epoch [2/5], Step [300/600], Loss: 0.0090
Epoch [2/5], Step [400/600], Loss: 0.0031
Epoch [2/5], Step [500/600], Loss: 0.0402
Epoch [2/5], Step [600/600], Loss: 0.0043
Accuracy of the network on the 10000 test images: 98.88 %
Epoch [3/5], Step [100/600], Loss: 0.0073
Epoch [3/5], Step [200/600], Loss: 0.0310
Epoch [3/5], Step [300/600], Loss: 0.0172
Epoch [3/5], Step [400/600], Loss: 0.0155
Epoch [3/5], Step [500/600], Loss: 0.0132
Epoch [3/5], Step [600/600], Loss: 0.0032
Accuracy of the network on the 10000 test images: 99.04 %
Epoch [4/5], Step [100/600], Loss: 0.0014
Epoch [4/5], Step [200/600],

In [15]:
ss = []
for name, p in model.named_parameters():
    ss.append(p.numel())
    print(name, p.shape, p.numel())
print(sum(ss))

conv1.weight torch.Size([20, 1, 5, 5]) 500
conv1.bias torch.Size([20]) 20
batchnorm1.weight torch.Size([20]) 20
batchnorm1.bias torch.Size([20]) 20
conv2.weight torch.Size([50, 20, 5, 5]) 25000
conv2.bias torch.Size([50]) 50
batchnorm2.weight torch.Size([50]) 50
batchnorm2.bias torch.Size([50]) 50
fc1.weight torch.Size([500, 800]) 400000
fc1.bias torch.Size([500]) 500
fc2.weight torch.Size([10, 500]) 5000
fc2.bias torch.Size([10]) 10
431220
